Guia para permitir acesso remoto ao google drive:

https://medium.com/@umdfirecoml/a-step-by-step-guide-on-how-to-download-your-google-drive-data-to-your-jupyter-notebook-using-the-52f4ce63c66c

Api foi criada em:

https://console.cloud.google.com/apis/credentials?project=mestrado-240317

In [1]:
!pip install oauth2client

In [2]:
from apiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools
obj = lambda: None
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090,5000], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive'
store = file.Storage('token.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_id.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)

In [3]:
DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))

In [4]:
import httplib2
import io
import os.path

from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload

def get_folder(name):
    results = DRIVE.files().list(
        pageSize=10,
        q=("name = '{0}'".format(name) +
           " and mimeType = 'application/vnd.google-apps.folder'"),
        corpora="user",
        fields="nextPageToken, files(id, name, webContentLink, " +
               "createdTime, modifiedTime)").execute()
    items = results.get('files', [])
    if not items:
        return None
    return items[0]

def get_folder_contents(folder_id):
    results = DRIVE.files().list(
        q=("'{0}' in parents".format(folder_id)),
        corpora="user",
        fields="nextPageToken, files(id, name, webContentLink, " +
               "createdTime, modifiedTime, size)").execute()
    items = results.get('files', [])
    if not items:
        print('No files found.')
    return items

In [5]:
#obtem pasta
folder = get_folder('datasets')
#obtem arquivos da pasta
folder_items = get_folder_contents(folder['id'])

#realiza download dos arquivos
for file in folder_items:
    # decide if you should download
    #my_download_function(url=item['webContentLink'])
    file_name = file['name']
    if os.path.exists(file_name):
        local_file_size = os.path.getsize(file_name)
        remote_file_size = file['size']
        #verifica se tamanhos são diferentes
        if int(local_file_size) != int(remote_file_size):
            print(f"Arquivo {file_name} com tamanho diferente - remoto: {remote_file_size} local: {local_file_size}")
        #else:
        #    print(f"Arquivo {file_name} ja existe na pasta.")
        
        continue
    request = DRIVE.files().get_media(fileId=file['id'])
    fh = io.FileIO(file['name'], mode='w')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print (f" Download - {file_name} {int(status.progress() * 100)}%.")
    
    
    
    

 Download - tab_inst_utr_11.csv 100%.
 Download - tab_inst_utr_10.csv 100%.
 Download - tab_inst_utr_09.csv 100%.
 Download - tab_inst_utr_08.csv 100%.
 Download - tab_inst_utr_07.csv 100%.
 Download - tab_inst_utr_06.csv 100%.
 Download - tab_inst_utr_05.csv 100%.
 Download - tab_inst_utr_04.csv 100%.
 Download - tab_inst_utr_03.csv 100%.
 Download - tab_inst_utr_02.csv 100%.
 Download - tab_inst_utr_01.csv 100%.


In [6]:
for file in folder_items:
    # decide if you should download
    #my_download_function(url=item['webContentLink'])
    file_name = file['name']
    if os.path.exists(file_name):
        local_file_size = os.path.getsize(file_name)
        remote_file_size = file['size']
        print(f"Arquivo {file_name} com tamanho diferente - remoto: {remote_file_size} local: {local_file_size}")

#request

Arquivo timeseries_elevatoria_13.csv com tamanho diferente - remoto: 275236731 local: 275236731
Arquivo timeseries_elevatoria_12.csv com tamanho diferente - remoto: 291323606 local: 291323606
Arquivo timeseries_elevatoria_11.csv com tamanho diferente - remoto: 296200849 local: 296200849
Arquivo timeseries_elevatoria_10.csv com tamanho diferente - remoto: 263864410 local: 263864410
Arquivo timeseries_elevatoria_9.csv com tamanho diferente - remoto: 259925936 local: 259925936
Arquivo timeseries_elevatoria_8.csv com tamanho diferente - remoto: 285752478 local: 285752478
Arquivo timeseries_elevatoria_7.csv com tamanho diferente - remoto: 183130783 local: 183130783
Arquivo timeseries_elevatoria_6.csv com tamanho diferente - remoto: 42757217 local: 42757217
Arquivo timeseries_elevatoria_5.csv com tamanho diferente - remoto: 80504512 local: 80504512
Arquivo timeseries_elevatoria_4.csv com tamanho diferente - remoto: 77123814 local: 77123814
Arquivo timeseries_elevatoria_3.csv com tamanho dife

In [14]:
for elv in range(1,31):
    results = DRIVE.files().list(
        q=("name = 'tab_inst_utr_{0:02d}.csv'".format(elv)),
        corpora="user",
        fields="nextPageToken, files(id, name, size)").execute()
    file = results['files'][0]
    file_name = file['name']
    if os.path.exists(file_name):
        local_file_size = os.path.getsize(file_name)
        remote_file_size = file['size']
        #verifica se tamanhos são diferentes
        if int(local_file_size) != int(remote_file_size):
            print(f"Arquivo {file_name} com tamanho diferente - remoto: {remote_file_size} local: {local_file_size}")
        #else:
        #    print(f"Arquivo {file_name} ja existe na pasta.")
        
        continue
    request = DRIVE.files().get_media(fileId=file['id'])
    fh = io.FileIO(file['name'], mode='w')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print (f" Download - {file_name} {int(status.progress() * 100)}%.")

In [5]:
for elv in range(14,31):
    file_name = "timeseries_elevatoria_{0:02d}_.csv".format(elv)
    folder_id ='1hImzcSND865nbjxUfqaUFFkmIn44C9zY'
    file_metadata = {
        'name': file_name,
        'parents': [folder_id]

    }
    media = MediaFileUpload(file_name,
                            mimetype='text/csv',
                            resumable=True)
    file = DRIVE.files().create(body=file_metadata,
                                media_body=media,
                                fields='id').execute()
    print ('File {} ID: {}'.format(file_name, file.get('id')))

File timeseries_elevatoria_14_.csv ID: 1C2B7-xPRxW-gpVqfp_mtiLwTa4S1sx1l
File timeseries_elevatoria_15_.csv ID: 1uuJwzAgFaO5Zbeuvstd_eIj7O3X5r9cD
File timeseries_elevatoria_16_.csv ID: 1kNuxoYtCz45jsDWi7X3bdRl42rC-WqxI
File timeseries_elevatoria_17_.csv ID: 1T886DjJ6Vj_MD1u3aWH6z17psQrjMLh_
File timeseries_elevatoria_18_.csv ID: 1uJYfB70dq4x6qGbTr6LREJFHalIcNnux
File timeseries_elevatoria_19_.csv ID: 1O-A99kFI11hdXcdMY-wlZ3lfDk0jLNLx
File timeseries_elevatoria_20_.csv ID: 16y1PAA90tcC-VaEmkmeJt20JHnCW6b_U
File timeseries_elevatoria_21_.csv ID: 1PS26otUU8KXp25D8Nr67S1kDDOz6P-ce
File timeseries_elevatoria_22_.csv ID: 1nBRyOdGeKdqsPU31EYnCP_2nXypU2yoh
File timeseries_elevatoria_23_.csv ID: 1MgHEX8k_zF4Iah2VuCC6Zfl1xJ3lMEwC
File timeseries_elevatoria_24_.csv ID: 11xFacqml3OkeUpuLKxNwnomqFogcabs_
File timeseries_elevatoria_25_.csv ID: 1YAOc6gOtwV8We3TeFfYrOSoygJijXsBc
File timeseries_elevatoria_26_.csv ID: 1o_HACiUcUlQGzTJVVmxJSqSnJXrN57Ag
File timeseries_elevatoria_27_.csv ID: 1-B-dbXg4C1x

In [22]:
for elv in range(14,31):
    results = DRIVE.files().list(
        q=("name = 'timeseries_elevatoria_{0}.csv'".format(elv)),
        corpora="user",
        fields="nextPageToken, files(id, name, size)").execute()
    file = results['files'][0]
    file_name = file['name']
    if os.path.exists(file_name):
        local_file_size = os.path.getsize(file_name)
        remote_file_size = file['size']
        #verifica se tamanhos são diferentes
        if int(local_file_size) != int(remote_file_size):
            print(f"Arquivo {file_name} com tamanho diferente - remoto: {remote_file_size} local: {local_file_size}")